Not the best option for this problem, because of the mapping from 3d input to 2d output 

In [ ]:
# general tools
import os
import sys
from glob import glob

# data tools
import re
import time
import h5py
import random
import numpy as np
from random import shuffle
from datetime import datetime, timedelta

#tf.config.run_functions_eagerly(True)

In [ ]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du
import model_utils as mu

In [ ]:
def CONV_stack(X, channel, kernel_size=3, stack_num=2, 
               dilation_rate=1, activation='ReLU', 
               batch_norm=False, name='conv_stack'):
    '''
    Stacked convolutional layers:
    (Convolutional layer --> batch normalization --> Activation)*stack_num
    
    CONV_stack(X, channel, kernel_size=3, stack_num=2, dilation_rate=1, activation='ReLU', 
               batch_norm=False, name='conv_stack')
    
    
    Input
    ----------
        X: input tensor.
        channel: number of convolution filters.
        kernel_size: size of 2-d convolution kernels.
        stack_num: number of stacked Conv2D-BN-Activation layers.
        dilation_rate: optional dilated convolution kernel.
        activation: one of the `tensorflow.keras.layers` interface, e.g., ReLU.
        batch_norm: True for batch normalization, False otherwise.
        name: prefix of the created keras layers.
        
    Output
    ----------
        X: output tensor
        
    '''
    
    bias_flag = not batch_norm
    
    # stacking Convolutional layers
    for i in range(stack_num):
        
        activation_func = eval(activation)
        
        # linear convolution
        X = Conv2D(channel, kernel_size, padding='same', use_bias=bias_flag, 
                   dilation_rate=dilation_rate, name='{}_{}'.format(name, i))(X)
        
        # batch normalization
        if batch_norm:
            X = BatchNormalization(axis=3, name='{}_{}_bn'.format(name, i))(X)
        
        # activation
        activation_func = eval(activation)
        X = activation_func(name='{}_{}_activation'.format(name, i))(X)
        
    return X

In [ ]:
batch_norm = True
bias_flag = False
filter_num = [128, 64, 32]
kernel_size = 3

In [1]:

input_tensor = keras.Input((None, None, None, 4, 15))

X_skip = []
depth_ = len(filter_num)

X = input_tensor
# stacked conv2d before downsampling
# linear convolution
X = Conv3D(filter_num[0], kernel_size, padding='same', use_bias=bias_flag,)(X)
# batch normalization
if batch_norm:
    X = keras.layers.BatchNormalization(axis=-1)(X)        
# activation
X = keras.layers.Activation("gelu")(X)

X_skip.append(X)

# downsampling blocks
for i, f in enumerate(filter_num[1:]):
    X = Conv3D(f, kernel_size, strides=(pool_size, pool_size, pool_size), 
               padding='valid', use_bias=bias_flag,)(X)
    # batch normalization
    if batch_norm:
        X = keras.layers.BatchNormalization(axis=3, name='{}_bn'.format(name))(X)
    # activation
    X = keras.layers.Activation("gelu")(X)
    
    for j in range(2):
    
        X = Conv3D(f, kernel_size, padding='same', use_bias=bias_flag,)(X)

        # batch normalization
        if batch_norm:
            X = keras.layers.BatchNormalization(axis=-1)(X)        
        # activation
        X = keras.layers.Activation("gelu")(X)
      
    X_skip.append(X)

# reverse indexing encoded feature maps
X_skip = X_skip[::-1]
# upsampling begins at the deepest available tensor
X = X_skip[0]
# other tensors are preserved for concatenation
X_decode = X_skip[1:]
depth_decode = len(X_decode)

filter_num_decode = filter_num[:-1][::-1]

# upsampling with concatenation
for i in range(depth_decode):
    
    X = Conv3DTranspose(filter_num_decode[i], kernel_size, strides=(pool_size, pool_size, pool_size), 
                        padding='same', name='{}_trans_conv'.format(name))(X)
        
    # batch normalization
    if batch_norm:
        X = keras.layers.BatchNormalization(axis=3, name='{}_bn'.format(name))(X)
            
    # activation
    X = keras.layers.Activation("gelu")(X)
    
    
    X = keras.layers.concatenate([X,]+[X_decode[i],], axis=-1)
    
    # Stacked convolutions after concatenation 
    for j in range(2):
    
        X = Conv3D(f, kernel_size, padding='same', use_bias=bias_flag,)(X)

        # batch normalization
        if batch_norm:
            X = keras.layers.BatchNormalization(axis=-1)(X)        
        # activation
        X = keras.layers.Activation("gelu")(X)
        
X = keras.layers.Conv3D(1, kernel_size, padding='same', use_bias=True, name=name)(X)
X = keras.layers.Activation('sigmoid', name='{}_activation'.format(name))(X)


model = Model(inputs=[input_tensor,], outputs=[X,], name='{}_model'.format(name))

NameError: name 'keras' is not defined